In [7]:
#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
#
#
#                                                  project 2 - Objecto em AR
#
#
#---------------------------------------------------------------------------------------------------------------------------------------------------------------------


import cv2 
import cv2.aruco as aruco 
import numpy as np
#import time 

self = 0 
# verify disponibilidade 

if not hasattr(cv2, 'aruco'):
    raise ImportError("Not found cv2.aruco")

print("Version openvc: ", cv2.__version__)

class Project2:

    def __init__(self):
        data = np.load("calibration_data.npz")
        self.aruco_dir = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_50)
        self.mtx = data['mtx']
        self.dist = data['dist']      # correscopende a matriz intrínseca M = K[R|t]
        self.marker_length = 0.03
        # obj vitor
        self.objpath = r"C:\Users\marce\OneDrive\Desktop\Masters_degree\2_Semestre\VARM\project_2\guy.obj"
        #self.objpath = r"C:\Users\marce\OneDrive\Desktop\Masters_degree\2_Semestre\VARM\project_2\pou-obj\source\pou\pou.obj"
        #self.texture_img = cv2.imread(r"C:\Users\marce\OneDrive\Desktop\Masters_degree\2_Semestre\VARM\project_2\pou-obj\textures\Sin_título.png", cv2.IMREAD_COLOR)
        #self.objpath = r"C:\Users\marce\OneDrive\Desktop\Masters_degree\2_Semestre\VARM\project_2\pearlturtle_v1_L2.123c8b254914-0c70-4cc6-bc03-8d58f36d90c5\pearlturtle_v1_L2.123c8b254914-0c70-4cc6-bc03-8d58f36d90c5\13103_pearlturtle_v1_l2.obj"
        #self.texture_img = cv2.imread(r"C:\Users\marce\OneDrive\Desktop\Masters_degree\2_Semestre\VARM\project_2\pearlturtle_v1_L2.123c8b254914-0c70-4cc6-bc03-8d58f36d90c5\pearlturtle_v1_L2.123c8b254914-0c70-4cc6-bc03-8d58f36d90c5\13103_pearlturtle_diffuse.jpg", cv2.IMREAD_COLOR)
        print(f"mtx: {self.mtx}")
        print(f"dist: {self.dist}")

   
    def gerar_Aruco(self):
        # gerando marcadores Aruco 

        # marcadores testes 5
        num_markers = 5 

        # tamanho da imagem do marcador (em pixels)
        marker_size = 200

        
        for marker_id in range(num_markers):
            # criar imagem do marcador 
            marker_img = cv2.aruco.generateImageMarker(self.aruco_dir, marker_id, marker_size)

            # salvando o tempplate 
            filename = f'marker_{marker_id}.png'
            cv2.imwrite(filename, marker_img)

            print(f"well done save: {filename}")

    
    def desenhar_objecto(self, frame, rvec, tvec, corners):
        size = self.marker_length
         
        # desenhando os pontos para forma geometrica
        p1 = (corners[0][0][0][0], corners[0][0][0][1])
        p2 = (corners[0][0][1][0], corners[0][0][1][1])
        p3 = (corners[0][0][2][0], corners[0][0][2][1])
        p4 = (corners[0][0][3][0], corners[0][0][3][1])
        # definindo a reta do desenho 
        b1, b2 = p1[0], p1[1]
        b3, b4 = p2[0], p2[1]
        b5, b6 = p3[0], p3[1]
        b7, b8 = p4[0], p4[1]

        # piramede 
        # parte inferior
        # calculando os pontos centrais entre (b1-b5) e (b3-b7)
        cv2.line(frame, (int(b1), int(b2)), (int(b3), int(b4)), (0, 255, 255), 3)
        cv2.line(frame, (int(b5), int(b6)), (int(b7), int(b8)), (0, 255, 255), 3)
        cv2.line(frame, (int(b1), int(b2)), (int(b7), int(b8)), (0, 255, 255), 3)
        cv2.line(frame, (int(b3), int(b4)), (int(b5), int(b6)), (0, 255, 255), 3)

        # corners 
        x1, y1 = (b1 + b5) // 2, (b2 + b6) // 2
        x2, y2 =  (b3 + b7) // 2, (b4 + b8) // 2
        # criando o "topo" da pirâmide deslocado para cima (y - 200)
        cv2.line(frame, (int(b1), int(b2)), (int(x1), int(y1 - 200)), (0, 255, 255), 3)
        cv2.line(frame, (int(b5), int(b6)), (int(x1), int(y1 - 200)), (0, 255, 255), 3)
        cv2.line(frame, (int(b3), int(b4)), (int(x1), int(y2 - 200)), (0, 255, 255), 3)
        cv2.line(frame, (int(b7), int(b8)), (int(x1), int(y2 - 200)), (0, 255, 255), 3)

        """# base do objecto 
        base = 0   # delimitação bordas
        height = size * 0.5
        # desenho parametros
        axis = np.float32([
            [0, 0, 0 ],
            [size, 0, 0],
            [size, size, 0],
            [0, size, 0],
            [size / 2, size / 2, height]
        ])

        imgpts, _ = cv2.projectPoints(axis, rvec, tvec, self.mtx, self.dist)
        imgpts = np.int32(imgpts).reshape(-1, 2)

        # base do objecto
        #cv2.drawContours(frame, [imgpts[:4]], -1, (0, 255, 0), 2)
        base_pts = np.array([imgpts[0], imgpts[1], imgpts[2], imgpts[3]])
        cv2.polylines(frame, [base_pts], isClosed=True, color=(0, 255, 255), thickness=2)

        # ligando forma geometrica
        for i in range(4):
            cv2.line(frame, tuple(imgpts[i]), tuple(imgpts[4]), (0, 255, 255), 3)"""

        
        return frame

    def carregar_obj(self, pathobj): 
        # lendo ficheiro .obj 
        vertices = []
        faces = []

        with open(pathobj, 'r') as f:
            for line in f:
                if line.startswith('v '):
                    parts = line.strip().split()
                    vertex = list(map(float, parts[1:4]))
                    vertices.append(vertex)
                elif line.startswith('f '):
                    parts = line.strip().split()
                    face = [int(p.split('/')[0]) - 1 for p in parts[1:]]   # 
                    faces.append(face)

        return np.array(vertices, dtype=np.float32), faces

    def carregar_obj_textura(self, pathobj): 
        # lendo ficheiro .obj 
        vertices = []
        faces = []
        texcoords = []

        with open(pathobj, 'r') as f:
            for line in f:
                if line.startswith('v '):
                    parts = line.strip().split()
                    vertex = list(map(float, parts[1:4]))
                    vertices.append(vertex)
                elif line.startswith('vt '):
                    parts = line.strip().split()
                    texcoord = list(map(float, parts[1:3]))
                    texcoords.append(texcoord)
                elif line.startswith('f '):
                    parts = line.strip().split()[1:]
                    face = []
                    for p in parts:
                        v_idx, vt_idx = map(lambda x: int(x) - 1, p.split('/')[:2])
                        face.append((v_idx, vt_idx))
                    faces.append(face)

                    
        return np.array(vertices, dtype=np.float32), np.array(texcoords, dtype=np.float32), faces
    
    def objecto_texturizado(self, frame, vertices, texcoords, faces, rvec, tvec):
        imgpts, _ = cv2.projectPoints(vertices, rvec, tvec, self.mtx, self.dist)
        imgpts = imgpts.reshape(-1, 2)

        h, w, _ = self.texture_img.shape

        for face in faces:
            if len(face) < 3:
                print("teste 333")
                continue
                                # apenas triângulos suportados neste exemplo

            pts_3d = np.array([imgpts[v_idx] for v_idx, _ in face], dtype=np.float32)
            pts_uv = np.array([[texcoords[vt_idx][0] * w, texcoords[vt_idx][1] * h] for _, vt_idx in face], dtype=np.float32)
            print(pts_3d, "teste")
            # Checar se é triângulo
            if len(pts_3d) == 3 and len(pts_uv) == 3:
                M = cv2.getAffineTransform(pts_uv, pts_3d)
                triangle = cv2.warpAffine(self.texture_img, M, (frame.shape[1], frame.shape[0]), None, flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_TRANSPARENT)
                
                # Máscara do triângulo para aplicar sem sobrescrever tudo
                mask = np.zeros_like(frame)
                cv2.fillConvexPoly(mask, pts_3d.astype(np.int32), (255, 255, 255))
                frame = np.where(mask == 0, frame, triangle)

        return frame


    def objecto_3d(self, frame, vertices, faces, rvec, tvec):

        #pou 
        #texture_img = cv2.imread(r"C:\Users\marce\OneDrive\Desktop\Masters_degree\2_Semestre\VARM\project_2\pou-obj\textures\Sin_título.png")
        # turtle 
        #texture_img = cv2.imread(r"C:\Users\marce\OneDrive\Desktop\Masters_degree\2_Semestre\VARM\project_2\pearlturtle_v1_L2.123c8b254914-0c70-4cc6-bc03-8d58f36d90c5\pearlturtle_v1_L2.123c8b254914-0c70-4cc6-bc03-8d58f36d90c5\13103_pearlturtle_diffuse.jpg")

        # projeta os vertices 
        imgpts, _ = cv2.projectPoints(vertices, rvec, tvec, self.mtx, self.dist)
        imgpts = imgpts.reshape(-1, 2).astype(int)

        # Remover valores NaN
        if np.isnan(imgpts).any():
            print("Vertices inválidos detectados.")
            return frame

        for face in faces:
            pts = np.array([imgpts[idx] for idx in face], dtype=np.int32) # array de array ## buscando os pontos e desenhando 
            #print(pts)
            # adcionando pontos de projetção 
            cv2.polylines(frame, [pts], isClosed=True, color=(0, 255, 255), thickness=1)
            #cv2.fillPoly(frame, [pts], color=(0, 255, 100))

            # adcionado textura projetada 
            #color = cv2.mean(texture_img)[0:3]
            #color = tuple(map(int, color))
            #cv2.fillConvexPoly(frame, pts, color=color)
        
        return frame

    def normalizar_obj(self, vertices, scale):
        # centralizando 
        centro = np.mean(vertices, axis=0)
        #[2]=centro[2] -10
        print(f"centro: {centro}")
        vertices -= centro            # alinhando no centro da marker , retirando a translação

        # equadrando no espaço 
        max_dim = np.max(np.linalg.norm(vertices, axis=1))
        vertices = (vertices / max_dim) * scale

        return vertices


    def main(self):
        # open camera 
        cap = cv2.VideoCapture(0)
        # define resolution
        #cap.set(3, 1280)
        #cap.set(4, 720)
        # carregando paramêtros 
        parameters = aruco.DetectorParameters()
        # deteção parametros 
        detect = aruco.ArucoDetector(self.aruco_dir, parameters)

        # carregando a camera 
        while True:
            # carregando a camera 
            ret, frame = cap.read()
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            if not ret:
                break

            # deteção marcadores 
            corners, ids, _ = detect.detectMarkers(gray)
            

            # marcadores selecionados 
            
            if ids is not None and len(ids) > 0:

                for i in range(len(ids)):
                    # estimar a pose da camera 
                    rvecs, tvecs, _ = aruco.estimatePoseSingleMarkers(corners[i], self.marker_length, self.mtx, self.dist)

                    # tratando possiveis erros
                    (rvecs - tvecs).any()

                    # definindo ID 
                    marker_id = ids[i][0]
                    
                    # delimitando bordas
                    cv2.aruco.drawDetectedMarkers(frame, corners)

                    # projectando eixos 
                    cv2.drawFrameAxes(frame, self.mtx, self.dist, rvecs, tvecs, self.marker_length)

                    # coordenada x
                    X = (corners[i][0][0][0] + corners[i][0][1][0] + corners[i][0][2][0] + corners[i][0][3][0]) / 4  # dividir cantos do x definir o ponto do meio 

                    # coordenadas Y
                    Y = (corners[i][0][0][1] + corners[i][0][1][1] + corners[i][0][2][1] + corners[i][0][3][1]) / 4  # dividir cantos do y


                    print(X)
                    print(Y)
                    #
                    # exibir id
                    cv2.putText(frame, "ID: " + str(marker_id), (int(X), int(Y)), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 255), 2)
                    
                    # desenhando a objectos simples (pirâmede) 
                    #frame = self.desenhar_objecto(frame, rvecs, tvecs, corners)

                    # carregando objecto 
                    self.vertices , self.faces = self.carregar_obj(self.objpath)
                    print(f"Vertices: {len(self.vertices)}")
                    print(f"faces: {self.faces}")

                    # objecto com texturas 
                    #self.vertices , self.faces, self.texcoords = self.carregar_obj_textura(self.objpath)


                    # normalizndo objecto 
                    self.vertices = self.normalizar_obj(self.vertices, scale=0.05)
                    
                    # desenhando objecto 3d 
                    frame = self.objecto_3d( 
                        frame, 
                        self.vertices, 
                        self.faces,
                        rvecs,
                        tvecs
                    )
                    """# desenhando objecto 3d com textura 
                    frame = self.objecto_texturizado( 
                        frame, 
                        self.vertices, 
                        self.texcoords,
                        self.faces,
                        rvecs,
                        tvecs
                    )"""

                    # mostrar vetor de translação Z (profundidade)
                    z = tvecs[i][0][2]  # acesso ao valor Z
                    print(f"ID {marker_id} - Distância Z: {z:.2f}m")

            cv2.imshow('ARUCO Detection', frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()


         


if __name__ == '__main__':

    exec = Project2()
    exec.main()
    #exec.gerar_Aruco()







Version openvc:  4.11.0
mtx: [[572.83075191   0.         350.28398836]
 [  0.         573.48244374 268.31448992]
 [  0.           0.           1.        ]]
dist: [[-0.10965985  0.15942934  0.01340477 -0.00419671 -0.56787964]]
455.25
414.0
Vertices: 114
faces: [[47, 42, 0, 10], [38, 41, 52, 51], [0, 1, 3, 2], [43, 45, 41, 38], [10, 0, 2, 12], [9, 8, 7, 6], [12, 2, 4, 14], [2, 3, 5, 4], [7, 17, 32, 30], [18, 9, 6, 16], [14, 4, 9, 18], [24, 23, 36, 37, 20], [5, 15, 19, 8], [8, 19, 17, 7], [1, 11, 13, 3], [3, 13, 15, 5], [46, 49, 45, 43], [44, 48, 53, 40], [39, 50, 11, 1], [22, 25, 21, 34, 35], [63, 62, 29, 26], [23, 22, 35, 36], [5, 8, 25, 22], [9, 4, 23, 24], [8, 9, 24, 25], [4, 5, 22, 23], [29, 28, 27, 26], [64, 63, 26, 27], [62, 65, 28, 29], [65, 64, 27, 28], [17, 16, 33, 32], [16, 6, 31, 33], [6, 7, 30, 31], [30, 32, 58, 59, 54, 55], [33, 31, 56, 57, 60, 61], [32, 33, 61, 58], [60, 57, 54, 59], [34, 37, 36, 35], [37, 34, 21, 20], [48, 44, 45, 49], [39, 42, 43, 38], [44, 40, 41, 45], [